Parameters

# Initialize

Parameters

In [ ]:
model_name="124M" # Options: 124M, 255M, 774M, 1558M
train_text_file="clean_rulebooks.txt"
model_save_name="Board-Test"

Imports

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

import numpy as np
import tensorflow as tf

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
!nvidia-smi

Sat Oct  2 18:28:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    59W / 149W |   1237MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

TPU Initialization

In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)  # Connects TPU.
tf.tpu.experimental.initialize_tpu_system(tpu)  # Initializes TPU.
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)  # TPU config of sorts.

AttributeError: ignored

Load GPT2

In [ ]:
gpt2.download_gpt2(model_name=model_name)
gpt2.mount_gdrive()

gpt2.copy_file_from_gdrive(train_text_file)

Fetching checkpoint: 1.05Mit [00:00, 219Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 8.37Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 625Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:15, 32.5Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 478Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 9.77Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 9.84Mit/s]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Finetune

Finetune Parameters

## Finetune Model

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=train_text_file,
              model_name=model_name,
              steps=501,
              restore_from='fresh',
              run_name=model_save_name,
              print_every=10,
              sample_every=20,
              save_every=100
              )

ValueError: ignored

## Save/Load Model to/from Google Drive

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name=model_save_name)

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name=model_save_name)

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=model_save_name)

Loading checkpoint checkpoint/run1/model-555
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-555


# Generate Text

## Generation Parameters

In [ ]:
gen_length = 250  # Tokens to generate (1023 max)
gen_temp = 0.7  # Higher temp, crazier text (Recommend between 0.7-1)
nsamples = 5  # Amount of texts to generate
batch_size = 5
# top_k = 40  # Limits crazy
# top_p = 0.9 # Limits cumulative probability

## Generation

### Test Generation

In [ ]:
# text = gpt2.generate(sess, length=gen_length, temperature=gen_temp, run_name=model_save_name)
text = gpt2.generate(sess, run_name=model_save_name)
# With prefix
# text = gp2.generate(sess, return_as_list=True, length=gen_length, temperature=gen_temp, prefix="The rules are", run_name=model_save_name)[0]

I do love thee, sir: would my heart were as yours.

ANTIGONUS:
You are a little worm, sir, but not a full worm.

SICINIUS:
You are a little worm, sir, but not a full worm.

SICINIUS:
You are a little worm, sir, but not a full worm.

SICINIUS:
Shepherd, I love thee not; I am not deceived.

CAPULET:
Not a shepherd, but a full-grown man: I love thee.

SICINIUS:
Shepherd, I love thee not; I am not deceived.

CAPULET:
Not a shepherd, but a full-grown man: I love thee.

SICINIUS:
Shepherd, I love thee not; I am not deceived.

CAPULET:
Shepherd, I love thee not; I am not deceived.

CAPULET:
Shepherd, thou hast loved me for a father; I am a son.

CAPULET:
Shepherd, I have a quarrel with a fiddler.

CAPULET:
I told thee how, soldier: now I see thy quarrel is not
so short. What! she hath deceived thee: I am a daughter.

SICINIUS:
Why, what?

CAPULET:
I told thee she hath deceived thee;
For thou wast the voice of a little dove,
Like to a winged tortoise,
Culling terror from the gentle sea.

SICIN

### Generate to file

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)